In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tensorflow as tf
import keras
import albumentations as A
from sklearn import model_selection, preprocessing 
import cv2
import tensorflow as tf
import numpy as np 
from matplotlib import pyplot as plt
from PIL import Image


# Loading Paths, Directory and Data-Folders 

In [ ]:
INPUT_PATH = "../input/cassava-leaf-disease-classification/"
train_images_path = INPUT_PATH+"train_images/"
test_images_path = INPUT_PATH+"test_images/"
sample = "../input/cassava-leaf-disease-classification/sample_submission.csv"

In [ ]:
df = pd.read_csv(INPUT_PATH+"train.csv")##../input/cassava-leaf-disease-classification/train.csv"
df.head(5)

In [ ]:
for img in os.listdir(INPUT_PATH+"train_images/")[:1]:
    #print(img)
    img = Image.open(os.path.join(train_images_path+img))
    plt.imshow(img)
    plt.show()

In [ ]:
sample_df = pd.read_csv(sample)
sample_df.head()

In [ ]:
num_classes = sorted(df["label"].unique())
df.info()

# **Prepare Dataset For training**

In [ ]:
df.label = df.label.astype("str")
batch_size=16
input_size = (299, 299)

**Data Augmentation**

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = 0.1,
    rotation_range=360,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    #channel_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None,
    )

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_images_path,
    x_col="image_id",
    y_col="label",
    batch_size=batch_size,
    target_size=input_size,
    class_mode="sparse", 
    subset = "training"
)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.1)
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_images_path,
    x_col="image_id",
    y_col="label",
    batch_size=batch_size,
    target_size=input_size ,
    class_mode="sparse", 
    subset="validation")

**Let's Have a look of few data** 

In [ ]:
plt.figure(figsize=(12,10))
for i in range(16):
    plt.subplot(4,4,i+1)
    batch = train_generator.next()
    image = batch[0].astype('uint8')
    plt.imshow(np.array(image[0,:,:,::-1]))
    plt.axis("off")
# show the figure
plt.show()            
                        

# Define Model, Train and Evaluate

In [ ]:
from keras import Model
from keras import optimizers
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Dense

model_dir = "../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
model = Sequential()
model.add( tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights= model_dir, input_shape=(299, 299, 3)))

model.add(GlobalAveragePooling2D())
model.add(Dense(len(num_classes), activation="softmax"))
model.summary()


In [ ]:

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
             metrics=['accuracy'])

**Let's Train**

In [ ]:

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath='best_inceptionresnetV2.h5',
        save_weights_only=True,
        monitor='val_loss',
        mode='max',
        save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

hist  = model.fit_generator(
    train_generator,
    validation_data = valid_generator,
    #
    steps_per_epoch = len(df)*0.9//batch_size,
    validation_steps = len(df)*0.1//batch_size, 
    epochs = 10,
    callbacks = [model_checkpoint ,early_stopping],
    
)

In [ ]:
# accuracy 
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
#model = model.load_weights("./best_inceptionresnetV2.h5")

**Predict and make Submission**

In [ ]:

predictions = []
for  image_id in sample_df.image_id:
    img = Image.open(os.path.join(test_images_path+image_id))
    img = img.resize((224,224))
    img = np.expand_dims(img, axis=0)
    predictions.append(np.argmax(model.predict(img)))

sample_df["label"] = predictions
sample_df

In [ ]:
sample_df.head


In [ ]:
sample_df.to_csv("submission.csv", index = False)